In [7]:
%pip install PyniteFEA[all]
%pip install Pynite
%pip install trimesh

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Below is the Code with point load on members

In [1]:
# ─────────────────────────────────────────────────────────────────
# 1) IMPORTS
# ─────────────────────────────────────────────────────────────────
from Pynite.Visualization import Renderer
from Pynite.FEModel3D import FEModel3D
import pandas as pd
import numpy as np

# ─────────────────────────────────────────────────────────────────
# 2) READ YOUR CSV FILES
# ─────────────────────────────────────────────────────────────────
nodes_df   = pd.read_csv(r'Documents/nodes.csv')
beams_df   = pd.read_csv(r'Documents/beams.csv')
columns_df = pd.read_csv(r'Documents/columns.csv')

# ─────────────────────────────────────────────────────────────────
# 3) INITIALIZE A NEW FINITE‐ELEMENT MODEL
# ─────────────────────────────────────────────────────────────────
model = FEModel3D()

# ─────────────────────────────────────────────────────────────────
# 4) ADD NODES TO THE MODEL
# ─────────────────────────────────────────────────────────────────
for _, row in nodes_df.iterrows():
    nid = str(row['ID'])
    model.add_node(nid, row['X'], row['Y'], row['Z'])

# ─────────────────────────────────────────────────────────────────
# 5) DEFINE MATERIAL & SECTION
# ─────────────────────────────────────────────────────────────────
# --- Define an RCC material instead of Steel ---
f_c  = 4.0            # concrete compressive strength (ksi)
E_c  = 57000 * (f_c**0.5)            # elastic modulus (ksi)
G_c  = E_c / (2*(1 + 0.2))           # shear modulus (ksi), nu ≈ 0.2
rho_c = 150/1728 * 1e-3              # density (kip/in³)

model.add_material('RCC', E_c, G_c, f_c, rho_c)

# --- Use that RCC material when you add members ---
# (you still define the section geometry the same way)
A, Iy, Iz, J = 8.84, 49.0, 14.4, 0.95
model.add_section('W14X30', A, Iy, Iz, J)

# ─────────────────────────────────────────────────────────────────
# 6) ADD MEMBERS (BEAMS + COLUMNS)
# ─────────────────────────────────────────────────────────────────
for _, row in beams_df.iterrows():
    bid = str(row['ID'])
    model.add_member(bid, str(row['i_node']), str(row['j_node']), 'RCC', 'W14X30')
for _, row in columns_df.iterrows():
    cid = str(row['ID'])
    model.add_member(cid, str(row['i_node']), str(row['j_node']), 'RCC', 'W14X30')

# ─────────────────────────────────────────────────────────────────
# 7) DEFINE SUPPORTS (Z=0 nodes fully fixed)
# ─────────────────────────────────────────────────────────────────
for _, row in nodes_df.iterrows():
    fixed = (row['Z'] == 0)
    model.def_support(
        str(row['ID']),
        fixed, fixed, fixed,   # Tx, Ty, Tz
        fixed, fixed, fixed    # Rx, Ry, Rz
    )

# ─────────────────────────────────────────────────────────────────
# 8) COMPUTE MEMBER LENGTHS
# ─────────────────────────────────────────────────────────────────
beam_lengths   = {}
column_lengths = {}
for _, row in beams_df.iterrows():
    b, ni, nj = str(row['ID']), str(row['i_node']), str(row['j_node'])
    xi, yi, zi = model.nodes[ni].X, model.nodes[ni].Y, model.nodes[ni].Z
    xj, yj, zj = model.nodes[nj].X, model.nodes[nj].Y, model.nodes[nj].Z
    beam_lengths[b] = float(np.sqrt((xj-xi)**2 + (yj-yi)**2 + (zj-zi)**2))

for _, row in columns_df.iterrows():
    c, ni, nj = str(row['ID']), str(row['i_node']), str(row['j_node'])
    xi, yi, zi = model.nodes[ni].X, model.nodes[ni].Y, model.nodes[ni].Z
    xj, yj, zj = model.nodes[nj].X, model.nodes[nj].Y, model.nodes[nj].Z
    column_lengths[c] = float(np.sqrt((xj-xi)**2 + (yj-yi)**2 + (zj-zi)**2))

# ─────────────────────────────────────────────────────────────────
# 9–10) APPLY UNIFORM DL & LL ON ALL MEMBERS
# ─────────────────────────────────────────────────────────────────
length_lookup = {**beam_lengths, **column_lengths}

for mid in model.members:
    # 1) Get member length
    if mid not in length_lookup:
        raise KeyError(f"No length for member '{mid}'")
    L = length_lookup[mid]

    # 2) Assign load intensities
    if mid in beam_lengths:
        w_DL, w_LL = -0.02, -0.015    # beam dead/Live loads
    else:
        w_DL, w_LL = -0.01, -0.008    # column dead/Live loads

    # 3) Compute equivalent point‐loads at midspan
    P_DL = w_DL * L
    P_LL = w_LL * L
    x_mid = L / 2.0

    # 4) Apply as point loads in global X direction
    model.add_member_pt_load(mid, 'FX', P_DL, x_mid, case='DL')
    model.add_member_pt_load(mid, 'FX', P_LL, x_mid, case='LL')


# ─────────────────────────────────────────────────────────────────
# 10.5) APPLY LATERAL NODE LOAD (“W”)
# ─────────────────────────────────────────────────────────────────
for nid in model.nodes:
    model.add_node_load(nid, 'FZ', -25, case='W')

# ─────────────────────────────────────────────────────────────────
# 11) DEFINE LOAD COMBINATIONS
# ─────────────────────────────────────────────────────────────────
# Note: using 'DL' to reference your dead‐load point‐loads,
#       and 'W' for the lateral node load
model.add_load_combo('0.9DL+1.0W', factors={'DL':0.9, 'W':1.0})
model.add_load_combo('1.2DL+1.0W', factors={'DL':1.2, 'W':1.0})

# ─────────────────────────────────────────────────────────────────
# 12) RUN YOUR ANALYSIS
# ─────────────────────────────────────────────────────────────────
model.analyze()

# ─────────────────────────────────────────────────────────────────
# 12.5) VISUALIZE DEFORMED SHAPE
# ─────────────────────────────────────────────────────────────────
rndr = Renderer(model)
rndr.annotation_size = .05
rndr.deformed_shape   = True
rndr.deformed_scale   = 500
rndr.combo_name       = '1.2DL+1.0W'
rndr.show_loads       = True
rndr.show_reactions   = True
rndr.window_width     = 750
rndr.window_height    = 750
rndr.render_model()




# ─────────────────────────────────────────────────────────────────
# (Right after you call model.analyze())
# ─────────────────────────────────────────────────────────────────
# from Pynite.Rendering import Renderer    # use the new Rendering module :contentReference[oaicite:0]{index=0}

# # Create a renderer for your entire model
# renderer = Renderer(model)

# # Match your single‐member example’s settings:
# renderer.annotation_size = 6       # text height
# renderer.deformed_shape   = True   # show deflected shape
# renderer.deformed_scale   = 100    # magnification factor
# renderer.render_loads     = True   # draw member + node loads
# renderer.render_reactions = True   # draw support reactions
# renderer.combo_name       = '1.2D+1.0W'
# renderer.window_width     = 750
# renderer.window_height    = 750

# # Pop up the interactive view
# renderer.render_model()

# ─────────────────────────────────────────────────────────────────
# 13) EXTRACT & PRINT GLOBAL STIFFNESS MATRIX [K]
# ─────────────────────────────────────────────────────────────────
K_sparse = model.K(combo_name='1.2D+1.0W', log=True, check_stability=True, sparse=True)
K_dense  = K_sparse.toarray()
print("Size:", K_dense.shape)
print("First 5×5 block:\n", K_dense[:5, :5])

labels = [f"{nid}_{dof}" for nid in model.nodes for dof in ('DX','DY','DZ','RX','RY','RZ')]
K_df = pd.DataFrame(K_dense, index=labels, columns=labels)
print("Global Stiffness Matrix [K]:")
print(K_df)

# ─────────────────────────────────────────────────────────────────
# 14) PRINT NODAL DISPLACEMENTS UNDER ULS
# ─────────────────────────────────────────────────────────────────
print("\nNODAL DISPLACEMENTS (under 1.2D+1.0W):")
for nid, node in model.nodes.items():
    dx = node.DX.get('1.2D+1.0W', 0.0)
    dy = node.DY.get('1.2D+1.0W', 0.0)
    dz = node.DZ.get('1.2D+1.0W', 0.0)
    print(f"  Node {nid}: DX={dx:.6e} in, DY={dy:.6e} in, DZ={dz:.6e} in")


- Adding nodal spring support stiffness terms to global stiffness matrix
- Adding spring stiffness terms to global stiffness matrix
- Adding member stiffness terms to global stiffness matrix


KeyError: '1.2D+1.0W'

In [15]:
# ─────────────────────────────────────────────────────────────────
# 1) IMPORTS
# ─────────────────────────────────────────────────────────────────
from Pynite.Visualization import Renderer
from Pynite.FEModel3D import FEModel3D
import pandas as pd
import numpy as np

# ─────────────────────────────────────────────────────────────────
# 2) READ YOUR CSV FILES
# ─────────────────────────────────────────────────────────────────
nodes_df   = pd.read_csv(r'Documents/nodes.csv')
beams_df   = pd.read_csv(r'Documents/beams.csv')
columns_df = pd.read_csv(r'Documents/columns.csv')

# ─────────────────────────────────────────────────────────────────
# 3) INITIALIZE A NEW FINITE‐ELEMENT MODEL
# ─────────────────────────────────────────────────────────────────
model = FEModel3D()

# ─────────────────────────────────────────────────────────────────
# 4) ADD NODES TO THE MODEL
# ─────────────────────────────────────────────────────────────────
for _, row in nodes_df.iterrows():
    nid = str(row['ID'])
    model.add_node(nid, row['X'], row['Y'], row['Z'])

# ─────────────────────────────────────────────────────────────────
# 5) DEFINE MATERIAL & SECTION (RCC)
# ─────────────────────────────────────────────────────────────────
f_c   = 4.0                           # concrete compressive strength (ksi)
E_c   = 57000 * (f_c ** 0.5)         # elastic modulus (ksi)
G_c   = E_c / (2 * (1 + 0.2))        # shear modulus (ksi), ν≈0.2
rho_c = (150 / 1728) * 1e-3          # density (kip/in³)

model.add_material('RCC', E_c, G_c, f_c, rho_c)

# Section geometry (all members use W14X30 here)
A, Iy, Iz, J = 8.84, 49.0, 14.4, 0.95
model.add_section('W14X30', A, Iy, Iz, J)

# ─────────────────────────────────────────────────────────────────
# 6) ADD MEMBERS (BEAMS + COLUMNS)
# ─────────────────────────────────────────────────────────────────
for _, row in beams_df.iterrows():
    bid = str(row['ID'])
    iN  = str(row['i_node'])
    jN  = str(row['j_node'])
    model.add_member(bid, iN, jN, 'RCC', 'W14X30')

for _, row in columns_df.iterrows():
    cid = str(row['ID'])
    iN  = str(row['i_node'])
    jN  = str(row['j_node'])
    model.add_member(cid, iN, jN, 'RCC', 'W14X30')

# ─────────────────────────────────────────────────────────────────
# 7) DEFINE SUPPORTS (Z=0 nodes fully fixed)
# ─────────────────────────────────────────────────────────────────
for _, row in nodes_df.iterrows():
    fixed = (row['Z'] == 0)
    model.def_support(
        str(row['ID']),
        fixed, fixed, fixed,   # Tx, Ty, Tz
        fixed, fixed, fixed    # Rx, Ry, Rz
    )

# ─────────────────────────────────────────────────────────────────
# 8) COMPUTE MEMBER LENGTHS
# ─────────────────────────────────────────────────────────────────
beam_lengths   = {}
column_lengths = {}

for _, row in beams_df.iterrows():
    mid = str(row['ID'])
    iN, jN = str(row['i_node']), str(row['j_node'])
    xi, yi, zi = model.nodes[iN].X, model.nodes[iN].Y, model.nodes[iN].Z
    xj, yj, zj = model.nodes[jN].X, model.nodes[jN].Y, model.nodes[jN].Z
    beam_lengths[mid] = float(np.sqrt((xj-xi)**2 + (yj-yi)**2 + (zj-zi)**2))

for _, row in columns_df.iterrows():
    mid = str(row['ID'])
    iN, jN = str(row['i_node']), str(row['j_node'])
    xi, yi, zi = model.nodes[iN].X, model.nodes[iN].Y, model.nodes[iN].Z
    xj, yj, zj = model.nodes[jN].X, model.nodes[jN].Y, model.nodes[jN].Z
    column_lengths[mid] = float(np.sqrt((xj-xi)**2 + (yj-yi)**2 + (zj-zi)**2))

# ─────────────────────────────────────────────────────────────────
# 9–10) APPLY EQUIVALENT POINT LOADS FOR DL & LL
# ─────────────────────────────────────────────────────────────────
length_lookup = {**beam_lengths, **column_lengths}

for mid in model.members:
    if mid not in length_lookup:
        raise KeyError(f"No length for member '{mid}'")
    L = length_lookup[mid]

    # Dead/Live intensities
    if mid in beam_lengths:
        w_DL, w_LL = -0.02, -0.015    # beams
    else:
        w_DL, w_LL = -0.01, -0.008    # columns

    # Equivalent midspan point‐loads
    P_DL  = w_DL * L
    P_LL  = w_LL * L
    x_mid = L / 2.0

    model.add_member_pt_load(mid, 'FX', P_DL,  x_mid, case='DL')
    model.add_member_pt_load(mid, 'FX', P_LL,  x_mid, case='LL')

# ─────────────────────────────────────────────────────────────────
# 10.5) APPLY LATERAL NODE LOAD (“W”)
# ─────────────────────────────────────────────────────────────────
for nid in model.nodes:
    model.add_node_load(nid, 'FY', -25, case='W')

# ─────────────────────────────────────────────────────────────────
# 11) DEFINE LOAD COMBINATIONS
# ─────────────────────────────────────────────────────────────────
model.add_load_combo('0.9DL+1.0W', factors={'DL': 0.9, 'W': 1.0})
model.add_load_combo('1.2DL+1.0W', factors={'DL': 1.2, 'W': 1.0})

# ─────────────────────────────────────────────────────────────────
# 12) RUN YOUR ANALYSIS
# ─────────────────────────────────────────────────────────────────
model.analyze()

# ─────────────────────────────────────────────────────────────────
# 12.5) VISUALIZE DEFORMED SHAPE
# ─────────────────────────────────────────────────────────────────
rndr = Renderer(model)
rndr.annotation_size = 0.05
rndr.deformed_shape   = False
rndr.deformed_scale   = 500
rndr.combo_name       = '1.2DL+1.0W'
rndr.show_loads       = True
rndr.show_reactions   = True
rndr.window_width     = 750
rndr.window_height    = 750
rndr.render_model()

# ─────────────────────────────────────────────────────────────────
# 13) COLUMN SLENDERNESS CHECK (L/r)
# ─────────────────────────────────────────────────────────────────
slenderness_limit_col = 200.0
print("\nCOLUMN SLENDERNESS CHECK (L/r):")
for _, row in columns_df.iterrows():
    cid = str(row['ID'])
    L   = column_lengths[cid]
    sec = model.sections['W14X30']
    r_y = np.sqrt(sec.Iy / sec.A)
    r_z = np.sqrt(sec.Iz / sec.A)
    r   = min(r_y, r_z)
    sl  = L / r
    status = "PASS" if sl <= slenderness_limit_col else "FAIL"
    print(f"  Column {cid}: L={L:.2f} in, r={r:.2f} in → L/r={sl:.1f} ({status})")

# ─────────────────────────────────────────────────────────────────
# 14) BEAM SLENDERNESS CHECK (L/r)
# ─────────────────────────────────────────────────────────────────
slenderness_limit_beam = 300.0
print("\nBEAM SLENDERNESS CHECK (L/r):")
for _, row in beams_df.iterrows():
    bid = str(row['ID'])
    L   = beam_lengths[bid]
    sec = model.sections['W14X30']
    r_y = np.sqrt(sec.Iy / sec.A)
    r_z = np.sqrt(sec.Iz / sec.A)
    r   = min(r_y, r_z)
    sl  = L / r
    status = "PASS" if sl <= slenderness_limit_beam else "FAIL"
    print(f"  Beam   {bid}: L={L:.2f} in, r={r:.2f} in → L/r={sl:.1f} ({status})")

# ─────────────────────────────────────────────────────────────────
# 15) PRINT GLOBAL STIFFNESS MATRIX & NODAL DISPLACEMENTS
# ─────────────────────────────────────────────────────────────────
K_sparse = model.K(combo_name='1.2DL+1.0W', log=True, check_stability=True, sparse=True)
K_dense  = K_sparse.toarray()
print("\nGlobal Stiffness Matrix [K] (size {}×{}):".format(*K_dense.shape))
labels = [f"{nid}_{dof}" for nid in model.nodes for dof in ('DX','DY','DZ','RX','RY','RZ')]
K_df = pd.DataFrame(K_dense, index=labels, columns=labels)
print(K_df)

print("\nNODAL DISPLACEMENTS (1.2DL+1.0W):")
for nid, node in model.nodes.items():
    dx = node.DX.get('1.2DL+1.0W', 0.0)
    dy = node.DY.get('1.2DL+1.0W', 0.0)
    dz = node.DZ.get('1.2DL+1.0W', 0.0)
    print(f"  Node {nid}: DX={dx:.6e} in, DY={dy:.6e} in, DZ={dz:.6e} in")



COLUMN SLENDERNESS CHECK (L/r):
  Column C0: L=2.50 in, r=1.28 in → L/r=2.0 (PASS)
  Column C1: L=2.50 in, r=1.28 in → L/r=2.0 (PASS)
  Column C2: L=2.50 in, r=1.28 in → L/r=2.0 (PASS)
  Column C3: L=2.50 in, r=1.28 in → L/r=2.0 (PASS)
  Column C4: L=2.50 in, r=1.28 in → L/r=2.0 (PASS)
  Column C5: L=2.50 in, r=1.28 in → L/r=2.0 (PASS)
  Column C6: L=2.50 in, r=1.28 in → L/r=2.0 (PASS)
  Column C7: L=2.50 in, r=1.28 in → L/r=2.0 (PASS)
  Column C8: L=2.50 in, r=1.28 in → L/r=2.0 (PASS)
  Column C9: L=2.50 in, r=1.28 in → L/r=2.0 (PASS)
  Column C10: L=2.50 in, r=1.28 in → L/r=2.0 (PASS)
  Column C11: L=2.50 in, r=1.28 in → L/r=2.0 (PASS)
  Column C12: L=2.50 in, r=1.28 in → L/r=2.0 (PASS)
  Column C13: L=2.50 in, r=1.28 in → L/r=2.0 (PASS)
  Column C14: L=2.50 in, r=1.28 in → L/r=2.0 (PASS)
  Column C15: L=2.50 in, r=1.28 in → L/r=2.0 (PASS)
  Column C16: L=2.50 in, r=1.28 in → L/r=2.0 (PASS)
  Column C17: L=2.50 in, r=1.28 in → L/r=2.0 (PASS)
  Column C18: L=2.50 in, r=1.28 in → L/r=